# Demonstration Notebook
## Train model to recognize notes from input sounds

By Ben Walsh \
For Bnaura

&copy; 2021 Ben Walsh <ben@bnaura.com>

## Contents

1. [Import Libraries](#lib_import)
1. [Import Data](#data_import)
1. [Evaluate Model](#model_eval)
1. [Demo Model](#model_demo) \
    4.1 [Pre-Recorded Audio](#pre_recorded_demo) \
    4.2 [Live Recording](#live_demo) 


TO DO
- C4 x 3 edge case debug
- note_len_time in melody_record can be float, not just int
- Move demo gui selection class (Note_sel_btn) to demo_util
- Adjust hum note list to files that exist - remove (Db, Eb ... B4?)
- ...
- Decouple load_training_data so X isn't needed for hum_len/feat_extract
- Submodule repo into simple_gui


In [1]:
%load_ext autoreload
%autoreload 2

## <a id = "lib_import"></a>1. Import Libraries

In [2]:
import sys
import time

import ipywidgets as widgets

import pickle

import xgboost as xgb
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

from scipy.io import wavfile as wav
from IPython.display import Audio

# Add custom modules to path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import util.music_util as mus 
from util.ml_util import feat_extract, load_training_data
from util import DATA_FOLDER, MODEL_FOLDER, AUDIO_FOLDER, SCALE


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
../1_audio\hum\Hum_Db4.wav does not exist
../1_audio\hum\Hum_Eb4.wav does not exist
../1_audio\hum\Hum_Gb4.wav does not exist
../1_audio\hum\Hum_Ab4.wav does not exist
../1_audio\hum\Hum_Bb4.wav does not exist
../1_audio\hum\Hum_B4.wav does not exist


## <a id = "data_import"></a>2.1 Import Data

### 2.1 Import Training Data (for reference)

In [3]:
X, _, fs = load_training_data(SCALE)

C:\Users\benja\Documents\Python\note-recognition\util\ml_util.py:124: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wav.read(training_data[note])


## <a id = "feat_save"></a> 2.2 Import Evaluation Data

In [4]:
X_test = pd.read_csv(os.path.join(DATA_FOLDER,"X_test.csv"))
y_test = pd.read_csv(os.path.join(DATA_FOLDER, "y_test.csv"))

# for label encoder
y_train = pd.read_csv(os.path.join(DATA_FOLDER,'y_train.csv'))

## <a id = "model_eval"></a>3. Evaluate Model

### 3.1 Load Model

In [5]:
latest_model = os.listdir(MODEL_FOLDER)[-1]
model_to_demo = pickle.load(open('{}/{}'.format(MODEL_FOLDER, latest_model), 'rb'))
print('Loading {} - {}'.format(latest_model,model_to_demo))

[13:21:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Loading model-2022-01-08-13-4-2-550101 - XGBRegressor(missing=nan)


### 3.2 Evaluation Model on Test Set

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_train)

C:\Users\benja\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [7]:
y_predict_xgb = model_to_demo.predict(X_test)
predictions = [round(value) for value in y_predict_xgb]
# evaluate predictions
print(f"Accuracy on test set: {100*accuracy_score(label_encoder.transform(y_test), predictions)}")

Accuracy on test set: 100.0


## <a id = "model_demo"></a>4. Demo Model

### <a id = "pre_recorded_demo"></a>4.1 Demo Model on Pre-Recorded File

In [8]:
hummed_note = 'C4' 
#hummed_note = 'E4' 
#hummed_note = 'A4' 
hum_wav_file = os.path.join(AUDIO_FOLDER, "hum","Hum_{}.wav".format(hummed_note))
fs_in, wav_sig_in = wav.read(hum_wav_file)

Audio(hum_wav_file)

C:\Users\benja\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """


### Predict note with trained model

In [9]:
# Extract features from test data

hum_len = X.shape[1]
hums = np.empty((1,hum_len))
hums[0,:] = wav_sig_in[:hum_len,1]

X_feat = mus.feat_extract(hums, fs_in, mus.note_to_freq, SCALE, debug=True)

Feature extraction processing time = 0.036


In [11]:
# Use loaded model to predict note
predictions = model_to_demo.predict(X_feat)
for prediction in predictions:
    # XGBoost outputs are class predictions, so use label_encoder inverse to translate to notes
    if isinstance(model_to_demo, xgb.sklearn.XGBRegressor):
        print('Predicted note: {}'.format(label_encoder.inverse_transform([round(prediction)])))
    else:
        print('Predicted note: {}'.format(prediction))

print('Input note note: {}'.format(hummed_note))


Predicted note: ['C4']
Input note note: C4


### Play back predicted note in piano

In [12]:
# XGBoost outputs are class predictions, so use label_encoder inverse to translate to notes
if isinstance(model_to_demo, xgb.sklearn.XGBRegressor):
    note_predict = mus.Note(note=label_encoder.inverse_transform([round(prediction)])[0])
else:
    note_predict = mus.Note(note=prediction)
note_predict.sound.play(0)

## Predict on length=3 melody 

### Concatenate wav files 

In [13]:
class Note_sel_btn:
        
    def __init__(self, scale, descr='Note 1'):
        self.scale = scale
        self.descr = descr
        self.btn = widgets.RadioButtons(
            options=scale,
            description=descr,
            disabled=False)

note1_sel = Note_sel_btn(scale=SCALE, descr='Note 1')
note2_sel = Note_sel_btn(scale=SCALE, descr='Note 2')
note3_sel = Note_sel_btn(scale=SCALE, descr='Note 3')


In [23]:
widgets.HBox(children=[note1_sel.btn, note2_sel.btn, note3_sel.btn])

In [24]:
MEL_NOTE_LIST = (note1_sel.btn.get_interact_value(), 
                 note2_sel.btn.get_interact_value(),
                 note3_sel.btn.get_interact_value()) 
MEL_FNAME = './melody_test.wav'
mel_sound = mus.Melody(MEL_NOTE_LIST, instr='hum', fname=MEL_FNAME)
Audio(MEL_FNAME)

### Extract signal and generate note predictions

In [25]:
fs, wav_signal = wav.read(MEL_FNAME)
DEBUG = False
predictions = mus.melody_transcribe(wav_signal, fs, model_to_demo, hum_len, SCALE, xgb_encoder=label_encoder, debug=DEBUG) 
print("Predicted notes: {}".format(predictions))

Predicted notes: ['E4' 'D4' 'C4']


### Generate Melody object 

In [26]:
# Create Melody object from predictions
melody_predict = mus.Melody(notes=predictions)
melody_predict.sound.play(0)

### <a id = "live_demo"></a> 4.2 Demo Model on Live Recording

In [30]:
REC_FILE_NAME = "./record_sound.wav"
NOTE_TOTAL = 1
NOTE_LEN_TIME = 2
DEBUG = False

In [31]:
melody_predict = mus.melody_rec_write(model=model_to_demo, 
                                  scale=SCALE, 
                                  note_total=NOTE_TOTAL, 
                                  note_len_time=NOTE_LEN_TIME, 
                                  file_name=REC_FILE_NAME, 
                                  xgb_encoder=label_encoder,
                                  debug=DEBUG)


--------
**------
****----
******--


In [32]:
melody_predict.sound.play(0)